# 国会議事録課題設定
2008年〜2018年分の国会の議事録用いた文書分類を実施する。
今回は2008〜2017年までの発言内容をもとに2018年の国会での発言内容がどの程度当てられるか検証する。

## 必要なライブラリ等のインストール

In [51]:
import glob
import re
import random
import pandas as pd
import numpy as np
import logging
from gensim.models import word2vec
from utils import normalize_text, text_to_words
from tqdm import tqdm_notebook
import pickle

## ファイルおよび必要な定数の定義
国会の議事録はinputディレクトリ以下に格納されている。  
・ディレクトリ形式  
&nbsp; input/議会名(衆議院or参議院or両院)/回次/議事録のtxt  
&nbsp; 議事録は日毎、会議ごとに保管されている。

    ex)
        20180122_本会議.txt  
        20180125_議院運営委員会.txt  
        20180125_本会議.txt  
        20180126_議院運営委員会.txt  

In [2]:
FILE_PATH_DIR = 'input/*/*/*.txt'
HOUSES, COMMITTEE, SPEAKER, DATE, SPEECH_ORDER, SPEECH_TEXT = 'houses', 'committee', 'speaker', 'date', 'speech_order', 'speech_text'
DATASET_FLAG, SPEECH_TO_WORDS = 'dataset_flag', 'speech_to_words'
file_path_list = glob.glob(FILE_PATH_DIR)

## 必要なファイルを読み込み整形する
議事録ファイルを1ファイルずつ読み込み、整形した上でリストに格納する。
リストには辞書形式で格納し、1つの辞書は会話単位とする。辞書の構成は  

'committee(会議名)', 'speaker(発言者)', 'date(日にち)', 'speech_order(発言順)', 'speech_text(発言内容)'

また、議事録ファイルは以下の構成が発言の数だけ並んでいる。

    会議名:
    発言順
    発言者:
    発言:

以下に実際の例を示す。

    ex)参議院　20180125_本会議.txt
    
    会議名:本会議
    発言順1
    発言者:伊達忠一
    発言:これより会議を開きます。
　    日程第一　国務大臣の演説に関する件（第二日）
　    去る二十二日の国務大臣の演説に対し、これより順次質疑を許します。大塚耕平君。   
　　     　〔大塚耕平君登壇、拍手〕

    会議名:本会議
    発言順2
    発言者:大塚耕平
    発言:民進党・新緑風会を代表して、政府四演説に対して総理に質問をいたします。・・・（以下発言内容）

    会議名:本会議
    発言順3
    発言者:安倍晋三
    発言:大塚耕平議員にお答えをいたします。・・・(以下発言内容)
    
    ・・・(以下発言が続く)
    
    会議名:本会議
    発言順8
    発言者:伊達忠一
    発言:御異議ないと認めます。
　     本日はこれにて散会いたします。
　　    　午前十一時三十六分散会

In [3]:
#random.shuffle(file_path_list)
speech_list = []
speech_flg = False
for file_path in tqdm_notebook(file_path_list):
    file_pattern = re.compile(r'^input/(.+)/.+/([0-9]{8}).+.txt$')
    date = file_pattern.sub( r'\2', file_path)
    houses = file_pattern.sub(r'\1', file_path)
    committee = ''
    speech_text = ''
    speech_order = ''
    speaker = ''
    with open(file_path, 'r') as f:
        for line in f:
            if not line.strip():
                continue
            if re.search(r'^発言者:', line):
                speech_flg = True
                speaker = re.sub(r'^発言者:', '', line)
                speaker = speaker.strip()
                continue
            elif re.search(r'^会議名:.+', line):
                speech_flg =  False
                if committee and speech_text:
                    #speech_list.append({COMMITTEE:committee, SPEAKER:speaker, DATE:date, SPEECH_ORDER:speech_order, SPEECH_TEXT:speech_text})
                    speech_list.append([houses, committee, speaker, date, int(speech_order), speech_text])
                    
                if not committee:
                    committee = re.sub(r'^会議名:', '', line)
                    committee = committee.strip()
                
                speech_text = ''
                continue
            elif re.search(r'^発言順.+', line):
                speech_order = re.sub(r'^発言順', '', line)
                speech_order = speech_order.strip()
                
            if speech_flg:
                line = re.sub(r'^発言:', '', line)
                line = line.replace('―', '')
                speech_text += line.strip()

## 課題
今回は、本会議と常任委員会について文書分類を実施する。  
対象は以下の通り。  
- 本会議
- 内閣委員会  
- 総務委員会  
- 法務委員会  
- 外務委員会  
- 財務金融委員会  
- 文部科学委員会  
- 厚生労働委員会  
- 農林水産委員会  
- 経済産業委員会  
- 国土交通委員会  
- 環境委員会  
- 安全保障委員会     
- 予算委員会  
- 決算行政監視委員会  
- 議院運営委員会   

懲罰委員会、国家基本政策委員会については発言数が少ないため今回は除外する。

また、それ以外の委員会については、単語の分散表現の作成のみに用いる

### 分類対象の抽出+形態素解析の実施
先ほどpandasの形式でまとめたデータの中から、文書分類に使用するデータセットの判別、および形態素解析を実施する。
#### 文書分類に使用するデータセットの判別
文書分類に使用するデータセットについては dataset_flagをTrue、しないものについてはFalseとして、pandasの形式に追加する。
#### 発言内容を形態素解析する
発言内容を形態素解析する。
今回は、発言内容から一部の品詞のみを抽出対象とする。  
抽出内容については下記の通り。

⚫️活用していない原型を用いる
- 形容詞
- 動詞

⚫️発言内容をそのまま用いる
- 名詞
- 副詞

形態素解析についてはのちに単語分散表現を作成するために、全ての発言について実施する。  
形態素解析を行った結果をspeech to wordカラムとして、pandasの形式に追加する。

In [4]:
classify_list = ['本会議', '内閣委員会', '総務委員会', '法務委員会', '外務委員会', '財務金融委員会', '文部科学委員会', '厚生労働委員会', '農林水産委員会', \
                '経済産業委員会', '国土交通委員会', '環境委員会', '安全保障委員会', '予算委員会', '決算行政監視委員会', '議院運営委員会']

speech_list = [speech +[ speech[1] in classify_list] + [text_to_words(speech[5])] for speech in tqdm_notebook(speech_list)]

In [10]:
 df_speech_list = pd.DataFrame(speech_list, columns=[HOUSES, COMMITTEE, SPEAKER, DATE, SPEECH_ORDER, SPEECH_TEXT, DATASET_FLAG, SPEECH_TO_WORDS])
df_speech_list.head()

,houses,committee,speaker,date,speech_order,speech_text,dataset_flag,speech_to_words
0,両院,国家基本政策委員会合同審査会,衛藤征士郎,20080109,1,これより国家基本政策委員会合同審査会を開会いたします。本日は、私が会長を務めさせていただきま...,False,これ 国家基本政策委員会 合同 審査 会 開会 いたす 。 本日 私 会長 務める させる ...
1,両院,国家基本政策委員会合同審査会,小沢一郎,20080109,2,総理、明けましておめでとうございます。新年早々から、総理にとっては余り気分のよくない、愉快で...,False,総理 明けましておめでとうございます 。 新年 早々 総理 余り 気分 よい 愉快 話 する...
2,両院,国家基本政策委員会合同審査会,福田康夫,20080109,3,冒頭、小沢代表から言われました、国民の立場に立った政治を進めなければいけない、私、全く同感で...,False,冒頭 小沢代表 言う れる 国民 立場 立つ 政治 進める いける 私 全く 同感 。 私 ...
3,両院,国家基本政策委員会合同審査会,小沢一郎,20080109,4,この問題は福田総理一人の責めに帰す話ではもちろんないんですけれども、しかしながら、今日こうい...,False,問題 福田総理 一人 責め 帰す 話 もちろん ない ん 今日 状況 起きる 現在 総理大臣...
4,両院,国家基本政策委員会合同審査会,福田康夫,20080109,5,ごもっともなお話でございまして、私もそう思っておりました。そこで、そこに至るところまでどうい...,False,ごもっとも お話 私 そう 思う おる 。 そこ 至る ところ ふう する 手順 少し 説明...


### pandas形式のモデルの保管

In [6]:
df_speech_list.to_csv('./data/speeches.csv', sep='\t')

## 単語分散表現の作成
2013〜2018年の議事録の発言内容をもとに、単語分散表現を作成する。  
単語分散表現は、分類対象外の会議も含めて作成する

In [14]:
speech_vec_text = [x[1][SPEECH_TO_WORDS] for x in tqdm_notebook(df_speech_list.iterrows())]
speech_vec_text = '\n'.join(speech_vec_text)
speech_vec_text = speech_vec_text.replace('。', '\n')
speech_vec_text = speech_vec_text.replace('、', ' ')

In [15]:
with open('data/speech_create_vec.txt', 'w') as f:
    f.write(speech_vec_text)

In [17]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
coupus = word2vec.Text8Corpus('data/speech_create_vec.txt')
basic_texts_model = word2vec.Word2Vec(coupus,size=200,min_count=5,window=5,iter=5)
basic_texts_model.save('data/basic_words_model.model')

## 分類対象の発言の抽出
分類対象の委員会のの発言を抽出する。  
2013〜2017年分は、学習用(および検証用)、2018年分についてはテスト用とする。  
抽出は以下の形式とする。

学習データはラベル(委員会名)、形態素解析済みの文章という形にまとめる。
発言については発言順ごとにマージする。

In [47]:
df_speech_list = pd.read_csv('./data/speeches.csv', sep='\t', header=0)

In [48]:
#dataset_flagがTrueの情報のみを抽出する
df_speech_list = df_speech_list[df_speech_list[DATASET_FLAG]]

df_speech_key = df_speech_list.loc[:, [HOUSES, COMMITTEE, DATE]]
df_speech_key = df_speech_key[~df_speech_key.duplicated()]
display(df_speech_key)

,houses,committee,date
1691,衆議院,本会議,20080111
1708,衆議院,総務委員会,20080115
1806,衆議院,議院運営委員会,20080111
1817,衆議院,厚生労働委員会,20080108
1990,衆議院,本会議,20080108
1998,衆議院,厚生労働委員会,20080111
2012,衆議院,財務金融委員会,20080115
2020,衆議院,議院運営委員会,20080108
2023,衆議院,議院運営委員会,20080117
2036,衆議院,法務委員会,20080115


### 学習用ファイルの作成
学習用ファイルの際にストップワードを削除する。
ストップワードについては下記サイトのものを使用。  
http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt

In [49]:
stop_words = []
with open('stopwords.txt', 'r') as f:
    stop_words = [line.strip() for line in f]

def delete_stopwords(text):
    text = text.split(' ')
    text = [x for x in text if ((not x in stop_words) and x)]
    text = ' '.join(text)
    return text
    
train_data = []
test_data = []
for keys in tqdm_notebook(df_speech_key.iterrows()):
    
    houses, committee, date = keys[1][HOUSES], keys[1][COMMITTEE], keys[1][DATE]
    year  = str(date)[0:4]
    df_search_result = df_speech_list[(df_speech_list[HOUSES] == houses) & (df_speech_list[COMMITTEE] == committee) & (df_speech_list[DATE] == date)].sort_index()
    speech_result = [str(x[1][SPEECH_TO_WORDS]).split('。') for x in df_search_result.iterrows()]
    speech_result   = [delete_stopwords(i) for j in speech_result for i in j if i]
    speech_result = '\n'.join(speech_result)
    if year == '2018':
        test_data.append([committee, speech_result])
    else:
         train_data.append([committee, speech_result])

In [52]:
with open('data/train_data.pickle', 'wb') as f:
    pickle.dump(train_data, f)
    
with open('data/test_data.pickle', 'wb') as f:
    pickle.dump(test_data, f)

In [53]:
train_data[0:2]

[['本会議',
  '会議 開く\n本日 参議院 本 院 送付 テロ対策海上阻止活動に対する補給支援活動の実施に関する特別措置法 案 否決 する 旨 通知 受領 する 返付 受ける\n憲法 条 項 基づく テロ対策海上阻止活動に対する補給支援活動の実施に関する特別措置法 案 院 議決 案 議題 する 直ちに 議決 する 動議 大島理森 君 提出\n大島理森 君 憲法 条 項 基づく テロ対策海上阻止活動に対する補給支援活動の実施に関する特別措置法 案 院 議決 案 議題 する 直ちに 議決 する 動議 提出 する れる おる\n動議 議題 いたす\n討論 通告 ある\n順次 許す\n仙谷由人 君\n仙谷由人 君 登壇\n民主党 仙谷由人\n民主党・無所属クラブ 代表 する 内閣 提出 テロ 新法 憲法 条 項 議決 する 動議 反対 討論 いたす\n拍手 条項 参議院 衆議院 可決 する 法案 異なる 議決 する 分の 多数 可決 できる 法律 成立 例外規定 置く いる\n我が国 憲法 体系 国会 二院制 とる いる 憲法 条 項 両院 可決 する 法律 なる 規定 本則 国会 一事不再議 原則 貫徹 する なる 五十 九条 項 使う 法律 成立 する せる する 極めて 異例 条 定める 両院協議会 協議 国会 提出 丁寧 審議 前置 する れる しゃにむに 押し切る 乱暴 運営 努める 回避 する\n五十 九条 項 あくまで 例外的 規定 の\n例外 権限 行使 する 正当 する 足りる 理由 必要 疑う 余地 ある\n参議院 意思 否定 する 衆議院 多数 強引 法律 する 緊急 する 国家 国民 回復 する がたい 危険 発生 する 損害 与える 具体 事由 ない 単に 形式的 付与 する れる いる 権限 濫用 する いる すぎる 憲法上 極めて 疑義 大きい いう 少なくとも 政治責任 厳しい 問う れる なる\n内閣 参議院 解散権 持つ いる 参議院 異なる 議決 内閣 与党 参議院 解散 する 制度 できる 程度 重要性 ある 初めて 議決 正当 する れる 考える\nわずか 参議院選挙 給油 継続 掲げる 自民党 公明党 歴史 敗北 いたす\n安倍内閣 思い込み 強い 法案 国会審議 空洞 する せる 次々 強行採決 する 国民 ノー 意思表

In [55]:
test_data[0:2]

[['議院運営委員会',
  '会議 開く\nまず 庶務 小 委員長 報告 発言 求める られる おる 許す\n石田真敏 君\n平成 年度 衆議院 歳出 予算 要求 庶務 小 委員会 審議 経過 結果 報告 申し上げる\n平成 年度 院 予定 経費 要求 額 余 前年度 予算額 比較 いたす 余 減額 なる おる\n情報システム 経費 給与 改定 伴う 人件費 増額 ある 一方 議員 経費 議員秘書 経費 議員会館 経費 減額\n概要 説明 申し上げる 国会 権能 行使 必要 経費 余 衆議院 運営 必要 経費 余 計上 いたす おる\n経費 議員 経費 事務局 法制局 事務 処理 する 必要 経費\n衆議院 施設 整備 必要 経費 余 民間 資金 活用 する 衆議院 施設 整備 必要 経費 余 計上 いたす おる\n経費 議事堂 本館 施設 整備 費 議員会館 整備 係る 不動産 購入 費\n国会 予備 必要 経費 計上 いたす おる\n平成 年度 衆議院 歳出 予算 要求 説明 申し上げる\n本件 つく 開く 委員会 審議 原案 決定 いたす 報告 いたす\n\n平成 年度 衆議院 予定 経費 要求 案 本号 末尾 掲載\n発言 求める られる おる 許す\n塩川鉄也 君\n〇 年度 本 院 予算 意見 述べる\n理由 本 院 予算 反対 表明 する\n第一 議員 経費\n委員長 手当 国会 役員 特別 する 特権 制度 廃止 求める\n〇 十一月 当時 民主党 国民新党 公明党 日本共産党 委員長 手当 廃止 法案 提出 委員会 審議 可決 する れる\n総選挙 廃案 なる 採決 反対 する 自民党 委員長 手当 廃止 反対 する いる 述べる いる\n委員長 手当 廃止 踏み出す\n文書通信交通滞在費 在京 議員 滞在 費 称す 手当 支給 する おる 国民 見る 合理的 説明 立つ 見直す\n情報 監視 審査会 関連 経費 計上 する れる いる\n国会 政府 秘密保護法 制 組み込む 国会 秘密 いう 国会法 基づく 体制 整備 継続 する\n国民 知る権利 侵害 する 国会 国政調査権 政府 監視 機能 制約 する 削除 する\n本 院 予算 かかわる 申し述べる\n憲政記念館 あり方\n憲政記念館 議会制民主主義 国民 理解 深める 憲政 資料 収集